<a href="https://colab.research.google.com/github/p-tech/BDAI-Files/blob/main/Crawl_SiteMap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import xml.etree.ElementTree as ET
import re

# Function to extract URLs from a sitemap
def get_urls_from_sitemap(sitemap_url):
    response = requests.get(sitemap_url)
    if response.status_code != 200:
        print(f"Failed to fetch sitemap: {response.status_code}")
        return []
    # Parse the sitemap
    root = ET.fromstring(response.content)
    namespaces = {'ns': 'http://www.sitemaps.org/schemas/sitemap/0.9'}
    urls = [url.text for url in root.findall('.//ns:loc', namespaces)]
    return urls

# Function to crawl a page and extract desired items
def crawl_page(url):
    proxy = {
    }

    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to fetch page {url}: {response.status_code}")
        return None
    soup = BeautifulSoup(response.content, 'html.parser')

    # Example: Extract title and meta description
    title = soup.title.string if soup.title else "No Title"
    meta_desc = soup.find('meta', attrs={'name': 'description'})
    meta_desc_content = meta_desc['content'] if meta_desc else "No Meta Description"

    h1 = soup.find('h1').get_text() if soup.find('h1') else ""
    h2 = soup.find('h2').get_text()  if soup.find('h2') else ""
    #price = soup.find('p', class_='property-price').get_text(strip=True)
    #price = re.sub(r'[^\d]', '', price)
    #price = re.search(r'\d[\d,]*', text).group().replace(',', '')
    #element = soup.find('div', class_='property-content')
    #if element:
    #  text_content = element.get_text(separator='\n', strip=True)  # Extracts text and removes leading/trailing whitespace
    #  print(text_content)
    #else:
    #  print("Element not found")

    return {"URL": url, "Title": title, "Meta Description": meta_desc_content, "H1": h1, "H2": h2 }

# Main script
def main():
    sitemap_url = "https://www.benburys.co.uk/sitemaps/properties.xml"  # Replace with your sitemap URL
    urls = get_urls_from_sitemap(sitemap_url)

    data = []
    capture = 0

    for url in urls:
        if capture < 5:
          extracted_data = crawl_page(url)
          if extracted_data:
              data.append(extracted_data)

          print(f'extracting data from {url}')

        capture = capture + 1


    # Save to DataFrame
    df = pd.DataFrame(data)
    # Save to CSV
    df.to_csv('extracted_data.csv', index=False)
    print("Data saved to extracted_data.csv")
    return df

# Run the script
if __name__ == "__main__":
    df = main()